Mengimport library yang dibutuhkan dan menautkan Google Drive



In [2]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Membuka dan memproses dataset

In [4]:
data = pd.read_excel('/content/drive/My Drive/PatternAnalysis/Online_Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Mengeksplorasi kolom pada dataset

In [5]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Mengeksplorasi region/negara yang unik di setiap transaksinya

In [6]:
data.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

Data cleaning

In [7]:
# Menghapus spasi tambahan pada deskripsi
data['Description'] = data['Description'].str.strip()
  
# Menghapus baris yang tidak terdapat nomor invoice
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')
  
# Menghapus transaksi yang dilakukan secara kredit
data = data[~data['InvoiceNo'].str.contains('C')]

Splitting data berdasarkan region/negara asal transaksi

In [8]:
# Transaksi yang dilakukan di Perancis / France
basket_France = (data[data['Country'] =="France"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

# Transaksi yang dilakukan di Inggris / United Kingdom
basket_UK = (data[data['Country'] =="United Kingdom"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

# Transaksi yang dilakukan di Portugis / Portugal
basket_Por = (data[data['Country'] =="Portugal"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))

# Transaksi yang dilakukan di Swedia / Sweden
basket_Sweden = (data[data['Country'] =="Sweden"]
		.groupby(['InvoiceNo', 'Description'])['Quantity']
		.sum().unstack().reset_index().fillna(0)
		.set_index('InvoiceNo'))


Data encoding

In [9]:
# Membuat hot encode untuk menyesuaikan dengan library yang digunakan
def hot_encode(x):
	if(x<= 0):
		return 0
	if(x>= 1):
		return 1

# Encoding dataset
basket_encoded = basket_France.applymap(hot_encode)
basket_France = basket_encoded

basket_encoded = basket_UK.applymap(hot_encode)
basket_UK = basket_encoded

basket_encoded = basket_Por.applymap(hot_encode)
basket_Por = basket_encoded

basket_encoded = basket_Sweden.applymap(hot_encode)
basket_Sweden = basket_encoded


Membuat model dan analisa hasil

Perancis / France

In [10]:
# Bentuk model
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)
  
# Mengumpulkan data pada sebuah dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                           antecedents  \
44                        (JUMBO BAG WOODLAND ANIMALS)   
258  (PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...   
272  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
301  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...   
302  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...   

                         consequents  antecedent support  consequent support  \
44                         (POSTAGE)            0.076531            0.765306   
258                        (POSTAGE)            0.051020            0.765306   
272                        (POSTAGE)            0.053571            0.765306   
301  (SET/6 RED SPOTTY PAPER PLATES)            0.102041            0.127551   
302    (SET/6 RED SPOTTY PAPER CUPS)            0.102041            0.137755   

      support  confidence      lift  leverage  conviction  
44   0.076531       1.000  1.306667  0.017961         inf  
258  0.051020       1.000  1.306667  0.011974     

Dari hasil diatas dapat terlihat bahwa cangkir kertas dan kertas, serta piring dibeli secara bersamaan di Perancis. Mengingat budaya Perancis yang sering berkumpul bersama keluarga setidaknya sekali dalam seminggu. Dan juga pemerintah Perancis telah melarang penggunaan plastik di negara tersebut, sehingga warga harus membeli dengan bahan kertas sebagai alternatif

Inggris / United Kingdom

In [11]:
# Bentuk model
frq_items = apriori(basket_UK, min_support = 0.01, use_colnames = True)

# Mengumpulkan data pada sebuah dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                                       antecedents             consequents  \
117           (BEADED CRYSTAL HEART PINK ON STICK)        (DOTCOM POSTAGE)   
2019  (JAM MAKING SET PRINTED, SUKI  SHOULDER BAG)        (DOTCOM POSTAGE)   
2296         (HERB MARKER MINT, HERB MARKER THYME)  (HERB MARKER ROSEMARY)   
2300   (HERB MARKER PARSLEY, HERB MARKER ROSEMARY)     (HERB MARKER THYME)   
2302      (HERB MARKER PARSLEY, HERB MARKER THYME)  (HERB MARKER ROSEMARY)   

      antecedent support  consequent support   support  confidence       lift  \
117             0.011036            0.037928  0.010768    0.975728  25.725872   
2019            0.011625            0.037928  0.011196    0.963134  25.393807   
2296            0.010714            0.012375  0.010232    0.955000  77.173095   
2300            0.011089            0.012321  0.010553    0.951691  77.240055   
2302            0.011089            0.012375  0.010553    0.951691  76.905682   

      leverage  conviction  
117   0.010349 

Jika aturan transaksi Inggris dianalisis lebih dalam, terlihat bahwa orang Inggris membeli piring teh berwarna berbeda secara bersamaan. Alasan di balik ini mungkin karena biasanya orang Inggris sangat menikmati teh dan sering mengumpulkan piring teh dengan warna berbeda untuk berbagai kesempatan.

Portugis / Portugal

In [12]:
# Bentuk model
frq_items = apriori(basket_Por, min_support = 0.05, use_colnames = True)

# Mengumpulkan data pada sebuah dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

                             antecedents                          consequents  \
1170  (SET 12 COLOUR PENCILS DOLLY GIRL)     (SET 12 COLOUR PENCILS SPACEBOY)   
1171    (SET 12 COLOUR PENCILS SPACEBOY)   (SET 12 COLOUR PENCILS DOLLY GIRL)   
1172  (SET 12 COLOUR PENCILS DOLLY GIRL)   (SET OF 4 KNICK KNACK TINS LONDON)   
1173  (SET OF 4 KNICK KNACK TINS LONDON)   (SET 12 COLOUR PENCILS DOLLY GIRL)   
1174  (SET 12 COLOUR PENCILS DOLLY GIRL)  (SET OF 4 KNICK KNACK TINS POPPIES)   

      antecedent support  consequent support   support  confidence       lift  \
1170            0.051724            0.051724  0.051724         1.0  19.333333   
1171            0.051724            0.051724  0.051724         1.0  19.333333   
1172            0.051724            0.051724  0.051724         1.0  19.333333   
1173            0.051724            0.051724  0.051724         1.0  19.333333   
1174            0.051724            0.051724  0.051724         1.0  19.333333   

      leverage  conviction

Saat menganalisis aturan asosiasi untuk transaksi Portugis, diamati bahwa set Tiffin (Knick Knack Tins) dan pensil warna. Kedua produk ini biasanya milik anak sekolah dasar. Kedua produk ini dibutuhkan oleh anak-anak di sekolah untuk membawa bekal dan untuk berkreasi masing-masing sehingga secara logis masuk akal untuk dipasangkan bersama.

Swedia / Sweden

In [13]:
# Bentuk model
frq_items = apriori(basket_Sweden, min_support = 0.05, use_colnames = True)

# Mengumpulkan data pada sebuah dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())


                           antecedents                     consequents  \
0        (12 PENCILS SMALL TUBE SKULL)   (PACK OF 72 SKULL CAKE CASES)   
1        (PACK OF 72 SKULL CAKE CASES)   (12 PENCILS SMALL TUBE SKULL)   
4       (ASSORTED BOTTLE TOP  MAGNETS)         (36 DOILIES DOLLY GIRL)   
5              (36 DOILIES DOLLY GIRL)  (ASSORTED BOTTLE TOP  MAGNETS)   
180  (CHILDRENS CUTLERY CIRCUS PARADE)  (CHILDRENS CUTLERY DOLLY GIRL)   

     antecedent support  consequent support   support  confidence  lift  \
0              0.055556            0.055556  0.055556         1.0  18.0   
1              0.055556            0.055556  0.055556         1.0  18.0   
4              0.055556            0.055556  0.055556         1.0  18.0   
5              0.055556            0.055556  0.055556         1.0  18.0   
180            0.055556            0.055556  0.055556         1.0  18.0   

     leverage  conviction  
0    0.052469         inf  
1    0.052469         inf  
4    0.052469       

Setelah menganalisis aturan di atas, ditemukan bahwa peralatan makan anak laki-laki dan perempuan dipasangkan bersama. Ini masuk akal karena ketika orang tua pergi berbelanja peralatan makan untuk anak-anaknya, dia ingin produknya sedikit disesuaikan dengan keinginan anak.